In [9]:
from QASMBench.interface.qiskit import QASMBenchmark

# path to the root directory of QASMBench
path = "QASMBench"

# selected category for QASMBench
category = "large" 

# select only the circuits with the number of qubits in the list
num_qubits_list = list(range(3, 100))

# whether to remove the final measurement in the circuit
remove_final_measurements = True

# whether to use qiskit.transpile() to transpile the circuits (note: must provide qiskit backend)
do_transpile = True  # Changed to True for transpiling circuits

# arguments for qiskit.transpile(). backend should be provided at least
transpile_args = { 'basis_gates': ['u', 'cp']}

bm = QASMBenchmark(path, category, num_qubits_list=num_qubits_list, do_transpile=do_transpile, **transpile_args, remove_final_measurements=remove_final_measurements)

In [8]:
circ_list = bm.circ_list



In [9]:
circ_name_list = bm.circ_name_list

In [10]:
for i in range(len(circ_list)):
    print(circ_name_list[i])
    print(circ_list[i].num_qubits)

dnn_n51
51
bwt_n57
57
square_root_n60
60
qft_n63
63
cc_n64
64
adder_n64
64
cat_n65
65
ising_n66
66
knn_n67
67
bv_n70
70
qugan_n71
71
multiplier_n75
75
wstate_n76
76
ghz_n78
78
swap_test_n83
83
bwt_n97
97
ising_n98
98


In [11]:
import os
import json
import numpy as np
from qiskit import transpile
from MLFM_GCP.graphs.GCP_hypergraph import QuantumCircuitHyperGraph
from MLFM_GCP.partitioning.FM.FM_main import *
from MLFM_GCP.partitioning.FM.multilevel_FM import *

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_FM_QASM_3_5it_larger.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_FM_QASM_3_5it_larger.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

num_partition_list = [2,3,4]

buffer = 0
for num_partitions in num_partition_list:
    for i, circ in enumerate(circ_list[buffer:]):
        # For each increase of 8 qubits, increase the number of partitions by 1
        i = i + buffer
        num_qubits = circ.num_qubits
        circuit = circ
        circuit = transpile(circuit, basis_gates = ['u', 'cp'])
        if circuit.depth() < 1000:
            print(f"Starting benchmark for {num_qubits} qubits, {num_partitions} partitions, circuit: {circ_name_list[i]}")

            base_graph = QuantumCircuitHyperGraph(circuit, group_gates=True)
            depth = base_graph.depth
            qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
            initial_assignment = set_initial_partitions(qpu_info, num_qubits, depth+2 ,num_partitions, reduced=True)
            num_qubits = circ.num_qubits
            # Create an All-to-All network
            
            

            
            iteration_data = []
            for iteration in range(5):
                
                # -------------------------
                # 1. Define/redefine circuit
                # -------------------------

                # base_graph = build_circuit(num_qubits,fraction=fraction,group_gates=True)
                # circuit = QAOA_random(num_qubits,prob=0.5, reps=1)

                
                # -------------------------
                # 5. Recursive refinement
                # -------------------------
                assignment_list_r, cost_list_r, time_list_r = MLFM_recursive(base_graph,
                                            initial_assignment,  
                                            qpu_info, 
                                            limit = 'qubit', 
                                            pass_list= None, 
                                            stochastic=True, 
                                            lock_nodes=False,
                                            log = False,
                                            add_initial = False,
                                            costs = None)
                
                total_time_r = sum(time_list_r)
                min_cost_r = min(cost_list_r)

                
                # -------------------------
                # 6. Store iteration-level results
                # -------------------------
                result_entry = {
                    "circuit_name": circ_name_list[i],
                    "num_qubits": num_qubits,
                    "num_partitions": num_partitions,
                    "iteration": iteration,
                    "r_cost":  min_cost_r,
                    "time_r": total_time_r,
                }
                
                detailed_results.append(result_entry)
                iteration_data.append(result_entry)
                
                # Update detailed JSON right away
                with open(detailed_filename, "w") as f:
                    json.dump(detailed_results, f, indent=2)
            
            # ---------------------------------------------------------------------
            # After 10 iterations, compute the means and log them
            # ---------------------------------------------------------------------

            r_cost_list = [x["r_cost"] for x in iteration_data]
            

            r_time_list = [x["time_r"] for x in iteration_data]
            

            mean_r_cost = float(np.mean(r_cost_list))
            

            mean_r_time = float(np.mean(r_time_list))
            
            # Print to console for quick logging
            print("=============================================")
            print(f"Finished 10 iterations for:")
            print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}, circuit: {circ_name_list[i]}")
            print("Mean Costs:")

            print(f"  Recursive:{mean_r_cost:.3f}")
            print("Mean Times (s):")

            print(f"  Recursive:{mean_r_time:.3f}")
            print("=============================================")
            
            # Store the aggregated means in a separate JSON
            mean_entry = {
                "circuit_name": circ_name_list[i],
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "mean_r_cost": mean_r_cost,
                "mean_r_time": mean_r_time,
            }
            
            mean_results.append(mean_entry)
            
            # Update the means JSON file
            with open(means_filename, "w") as f:
                json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Starting benchmark for 51 qubits, 2 partitions, circuit: dnn_n51
Finished 10 iterations for:
  # Qubits: 51, # Partitions: 2, circuit: dnn_n51
Mean Costs:
  Recursive:5.600
Mean Times (s):
  Recursive:7.516
Starting benchmark for 63 qubits, 2 partitions, circuit: qft_n63
Finished 10 iterations for:
  # Qubits: 63, # Partitions: 2, circuit: qft_n63
Mean Costs:
  Recursive:31.000
Mean Times (s):
  Recursive:38.323
Starting benchmark for 64 qubits, 2 partitions, circuit: cc_n64
Finished 10 iterations for:
  # Qubits: 64, # Partitions: 2, circuit: cc_n64
Mean Costs:
  Recursive:2.800
Mean Times (s):
  Recursive:11.651
Starting benchmark for 64 qubits, 2 partitions, circuit: adder_n64
Finished 10 iterations for:
  # Qubits: 64, # Partitions: 2, circuit: adder_n64
Mean Costs:
  Recursive:3.000
Mean Times (s):
  Recursive:17.994
Starting benchmark for 65 qubits, 2 partitions, circuit: cat_n65
Finished 10 iterations for:
  # Qubits: 65, # Partitions: 2, circuit: cat_n65
Mean Costs:
  Recursive

In [10]:
circ = bm.get('cc_n32')

In [ ]:
from MLFM_GCP.utils.qiskit_to_op_list import *

op_list = circuit_to_gate_layers(circ)

for layer in op_list:
    for op in layer:
        print(op)

print(circ)

['u', [0], ['q0'], [1.5707963267948966, 0.0, 3.141592653589793]]
['u', [1], ['q0'], [1.5707963267948966, 0.0, 3.141592653589793]]
['u', [2], ['q0'], [1.5707963267948966, 0.0, 3.141592653589793]]
['u', [3], ['q0'], [1.5707963267948966, 0.0, 3.141592653589793]]
['u', [4], ['q0'], [1.5707963267948966, 0.0, 3.141592653589793]]
['u', [5], ['q0'], [1.5707963267948966, 0.0, 3.141592653589793]]
['u', [6], ['q0'], [1.5707963267948966, 0.0, 3.141592653589793]]
['u', [7], ['q0'], [1.5707963267948966, 0.0, 3.141592653589793]]
['u', [8], ['q0'], [1.5707963267948966, 0.0, 3.141592653589793]]
['u', [9], ['q0'], [1.5707963267948966, 0.0, 3.141592653589793]]
['u', [10], ['q0'], [1.5707963267948966, 0.0, 3.141592653589793]]
['u', [11], ['q0'], [1.5707963267948966, 0.0, 3.141592653589793]]
['u', [12], ['q0'], [1.5707963267948966, 0.0, 3.141592653589793]]
['u', [13], ['q0'], [1.5707963267948966, 0.0, 3.141592653589793]]
['u', [14], ['q0'], [1.5707963267948966, 0.0, 3.141592653589793]]
['u', [15], ['q0'], 